# NYC Airbnb Price Prediction - Data Preparation

Use dataset published by Kaggle - https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data - to train a simple deep learning model to predict prices for Airbnb properties.


This notebook contains the common data loading and preparation steps:
- load data from the input CSV
- do an assessment of the dataset to understand the number of distinct, missing, or invalid values by column


# Common imports and variables
Imports and variable definitions that are common to the entire notebook


In [3]:
!pip install requests
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import datetime as dt
# common imports
import zipfile
import time
# import datetime, timedelta
import datetime
from datetime import datetime, timedelta
from datetime import date
from dateutil import relativedelta
from io import StringIO
import pandas as pd
import pickle
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from io import StringIO
import requests
import json
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline 
import os
import math
from subprocess import check_output
from IPython.display import display
import logging
import yaml
from collections import Counter
import re
import os
colab_path = "machine_learning_tabular_book/code/xgboost_basics"


In [22]:
!pwd

/content


In [5]:
def get_config(config_file):
    ''' open config file with name config_file that contains parameters
    for this module and return Python object

    Args:
        config_file: filename containing config parameters

    Returns:
        config: Python dictionary with config parms from config file - dictionary


    '''
    current_path = os.getcwd()
    path_to_yaml = os.path.join(current_path, config_file)
    print("path_to_yaml " + path_to_yaml)
    try:
        with open(path_to_yaml, 'r') as c_file:
            config = yaml.safe_load(c_file)
        return config
    except Exception as error:
        print('Error reading the config file ' + str(error))

In [6]:
def print_config_values(config):
    for val in config:
        print("config value ",val," ",str(config[val]))

In [27]:
def set_current_directory():
  ''' check to see if the notebook is being run in Colab, and if so, set the current directory appropriately'''
  if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive/machine_learning_tabular_book/code/xgboost_basics/notebooks


# Load Data
- ingest CSV into a Pandas dataframe 

In [7]:
def get_path():
    ''' get the path for data files

    Returns:
        path: path for data directory

    '''
    rawpath = os.getcwd()
    # data is in a directory called "data" that is a sibling to the directory
    # containing the notebook
    path = os.path.abspath(os.path.join(rawpath, '..', 'data'))
    return path

In [8]:
def ingest_data(path,input_csv,pickled_input_dataframe,save_raw_dataframe,load_from_scratch):
    ''' load data into dataframe
    Args:
        path: path containing input file
        input_csv: input file name
        pickled_input_dataframe: pickled version of input file

    Returns:
        path: path for data directory
    '''
    if load_from_scratch:
        # if loading from scratch, the raw CSV file is expected to be in the data directory which is a sibling to the 
        # directory that contains this notebook
        unpickled_df = pd.read_csv(os.path.join(path,input_csv)) 
        if save_raw_dataframe:
            file_name = os.path.join(path,pickled_input_dataframe)
            print("file_name is ",file_name)
            unpickled_df.to_pickle(file_name)
    else:
        unpickled_df = pd.read_pickle(os.path.join(path,pickled_input_dataframe))
        logging.debug("reloader done")
    return(unpickled_df)

# Assess values
- assess columns for missing or invalid values

In [9]:
def not_in_list(x, list):
    ''' check if a value is in a list
    Args:
        x: value to check
        list: list in which to check for the value

    Returns:
        retur_val: 1 if value is in not in list, 0 otherwise
    '''
    if x in list:
        return_val = 0
    else:
        return_val = 1
    return(return_val)
    

In [10]:
def neg_val(x):
    ''' check if a value is in a list
    Args:
        x: value to check
    
    Returns:
        retur_val: 1 if value is negative, 0 otherwise
    '''
    if x >= 0:
        return_val = 0
    else:
        return_val = 1
    return(return_val)

In [11]:
def basic_assessment(df,columns,valid_values,non_neg_continuous):
    ''' assess the values in a dataframe
    Args:
        df: dataframe for assessment
        columns: dictionary of column names by category
        valid_values: dictionary of valid values for categorical columns with limited number of valid values
        non_neg_continuous: list of continuous columns with only non-negative values as valid
    '''
    for col in list(df):
        print("Missing values in ",col," ",str(df[col].isna().sum()))
        print("Distinct values in ",col," ",str(df[col].nunique()))
    # for categorical columns with a limited number of valid values, count the number of invalid values by column
    for col in valid_values:
        print("non-valid values in column ",col," ",str(df[col].apply(lambda x:not_in_list(x,valid_values[col])).sum()))
    # count non-numeric values in continuous columns
    for col in columns['continuous']:
        # mask = pd.to_numeric(df['Hours_Worked'], errors='coerce').isna()
        mask = pd.to_numeric(df[col], errors='coerce').isna()
        print("non-numeric values in continuous col ",col," ",str(mask.sum()))
        # if there are no non-numeric values in the column and it muast have non-negative values, count negative values
        if (mask.sum()==0) and (col in non_neg_continuous):
            print("negative values in colum ",col," ",str(df[col].apply(lambda x:neg_val(x)).sum()))
    

In [12]:
def out_of_range(x,max,min):
    ''' count whether a value is in a range
    Args:
        x: value to check in range
        max: top of the range to check
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''
    if x > max or x < min:
        return_val = 1
    else:
        return_val = 0
    return(return_val)

In [13]:
def out_of_bounding_box(latitude,longitude,bounding_box):
    ''' count whether a location is within a bounding box
    Args:
        latitude: latitude portion of location
        longitude: longitude portion of location
        bounding_box: dictionary with max and min values to compare the location with
        min: bottom of the range to check
        
    Returns:
        ret_val: 1 if out of range, 0 otherwise
    '''    
    if ((latitude <= bounding_box['max_lat']) and (latitude >= bounding_box['min_lat'])) \
    and ((longitude <= bounding_box['max_long']) and (longitude >= bounding_box['min_long'])):
         ret_val = 0
    else:
         ret_val = 1
    return(ret_val)

In [14]:
def geo_assessment(df,bounding_box):
    ''' assess the geo columns in a dataframe by counting how many latitude and longitude values are outside the bounding box
    Args:
        df: dataframe for assessment
        bounding_box: dictionary of maximum and minimum valid latitude and longitude values
    ''' 
    # count the number of entries in the latitude column that are above or below a given amount
    # df['col_3'] = df.apply(lambda x: get_sublist(x.col_1, x.col_2), axis=1)
    print("latitude out of bounds count ",str(df['latitude'].apply(lambda x:out_of_range(x,bounding_box['max_lat'],bounding_box['min_lat'])).sum()))
    print("longitude out of bounds count ",str(df['longitude'].apply(lambda x:out_of_range(x,bounding_box['max_long'],bounding_box['min_long'])).sum()))
    print("location out of bounds count ",str(df.apply(lambda x: out_of_bounding_box(x.latitude,x.longitude,bounding_box), axis=1).sum()))

# Master cell
This cell contains calls to the other functions in this notebook to complete the data preparation

In [26]:
# master cell to call the other functions
# get the path for data files
set_current_directory()
path = get_path()
print("path is ",path)
config = get_config('data_preparation_config.yml')
logging.getLogger().setLevel(logging.WARNING)
logging.warning("logging check")
#print_config_values(config)
# load dataframe and, if parameter set, save CSV file as a pickled dataframe
df = ingest_data(path,config['file_names']['input_csv'],config['file_names']['pickled_input_dataframe'],config['general']['save_raw_dataframe'],config['general']['load_from_scratch'])
# get basic assessment information for the dataframe
basic_assessment(df,config['columns'],config['valid_values'],config['non_negative_continuous'])
# get assessment for geospatial information
geo_assessment(df,config['bounding_box'])
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/machine_learning_tabular_book/code/xgboost_basics/notebooks'
/content/drive/MyDrive/machine_learning_tabular_book/code/xgboost_basics
path is  /content/drive/My Drive/machine_learning_tabular_book/code/data
path_to_yaml /content/drive/My Drive/machine_learning_tabular_book/code/xgboost_basics/data_preparation_config.yml
Error reading the config file [Errno 2] No such file or directory: '/content/drive/My Drive/machine_learning_tabular_book/code/xgboost_basics/data_preparation_config.yml'


TypeError: ignored